In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [3]:
df = pd.read_csv('game.csv')
df.head()

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away
0,21946,1610610035,HUS,Toronto Huskies,24600001,1946-11-01 00:00:00,HUS vs. NYK,L,0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68,2,0
1,21946,1610610034,BOM,St. Louis Bombers,24600003,1946-11-02 00:00:00,BOM vs. PIT,W,0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,25.0,51,-5,0
2,21946,1610610032,PRO,Providence Steamrollers,24600002,1946-11-02 00:00:00,PRO vs. BOS,W,0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,-6,0
3,21946,1610610025,CHS,Chicago Stags,24600004,1946-11-02 00:00:00,CHS vs. NYK,W,0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,22.0,47,-16,0
4,21946,1610610028,DEF,Detroit Falcons,24600005,1946-11-02 00:00:00,DEF vs. WAS,L,0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,17,0


Filtering to games only in the last 2 years because teams change and previous games would not represent current teams

In [4]:
df['game_date'] = pd.to_datetime(df['game_date'])
df_filtered = df[df['game_date'] > '2021-01-01']
df_filtered.head()


,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away
59116,22020,1610612755,PHI,Philadelphia 76ers,22000082,2021-01-02,PHI vs. CHA,W,240,47.0,...,26.0,40.0,34.0,15.0,5.0,12.0,18.0,112,-15,1
59117,22020,1610612740,NOP,New Orleans Pelicans,22000084,2021-01-02,NOP vs. TOR,W,240,35.0,...,30.0,40.0,21.0,6.0,4.0,10.0,29.0,116,-4,1
59118,22020,1610612745,HOU,Houston Rockets,22000079,2021-01-02,HOU vs. SAC,W,240,36.0,...,33.0,47.0,11.0,5.0,4.0,15.0,21.0,94,-8,1
59119,22020,1610612754,IND,Indiana Pacers,22000080,2021-01-02,IND vs. NYK,L,240,34.0,...,41.0,51.0,23.0,4.0,3.0,16.0,16.0,106,4,1
59120,22020,1610612753,ORL,Orlando Magic,22000081,2021-01-02,ORL vs. OKC,L,240,40.0,...,46.0,52.0,22.0,5.0,5.0,12.0,17.0,108,9,1


Filtering out columns we do not need and renaming the columns we need

In [5]:
df_filtered['away'] = df_filtered['matchup_home'][7:10]
for idx, row in df_filtered.iterrows():
    df_filtered.loc[idx, 'away'] = row['matchup_home'][7:]

df_data = df_filtered[['team_abbreviation_home', 'away', 'game_date', 'wl_home']]
df_data.rename(columns={'team_abbreviation_home':'home', "away":"away", "game_date":"date", "wl_home":"wl_home"}, inplace=True)
df_data.head()

/var/folders/j4/xht4swnx3v38mbt0w75bjqs40000gn/T/ipykernel_43340/3551984928.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['away'] = df_filtered['matchup_home'][7:10]
/var/folders/j4/xht4swnx3v38mbt0w75bjqs40000gn/T/ipykernel_43340/3551984928.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data.rename(columns={'team_abbreviation_home':'home', "away":"away", "game_date":"date", "wl_home":"wl_home"}, inplace=True)


,home,away,date,wl_home
59116,PHI,CHA,2021-01-02,W
59117,NOP,TOR,2021-01-02,W
59118,HOU,SAC,2021-01-02,W
59119,IND,NYK,2021-01-02,L
59120,ORL,OKC,2021-01-02,L


Found 6 missing values on March 5, 2023
Game results are manually searched up and added

In [6]:
df_data.at[62312,'wl_home']='L'
df_data.at[62313,'wl_home']='W'
df_data.at[62314,'wl_home']='W'
df_data.at[62315,'wl_home']='L'
df_data.at[62316,'wl_home']='L'
df_data.at[62317,'wl_home']='W'
df_data[df_data.isnull().any(axis=1)]



,home,away,date,wl_home


Look for dupicate data, found none

In [7]:
dups = df_data.duplicated()
dups.any()

False

Baseline frequency

In [8]:
df_data.wl_home.value_counts(normalize=True)

W    0.558905
L    0.441095
Name: wl_home, dtype: float64

Pipeline

In [9]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
ohe = OneHotEncoder(sparse=False)

In [16]:
features = ['home', 'away']#, 'date']
X = df_data[features]
y = df_data.wl_home

In [12]:
ct = make_column_transformer(
       (ohe, ['home', 'away', 'date']),
       remainder = 'passthrough'
)
a = ct.fit_transform(X)
a[:5, : ]

/Users/joshhuang/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
X.head(5)

,home,away
59116,PHI,CHA
59117,NOP,TOR
59118,HOU,SAC
59119,IND,NYK
59120,ORL,OKC


Cross Validation

In [17]:
from sklearn.pipeline import make_pipeline

In [18]:
from sklearn.linear_model import LogisticRegression

ct = make_column_transformer(
       (ohe, ['home', 'away']),
       remainder = 'passthrough'
)

lgr = LogisticRegression()

p1 = make_pipeline(ct, lgr)

X = df_data[features]
y = df_data.wl_home


from sklearn.model_selection import cross_val_score

cross_val_score( p1, X , y, cv=5, scoring='accuracy').mean()

/Users/joshhuang/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/joshhuang/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/joshhuang/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/joshhuang/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was rename

0.6238123596833274